# SMLデータを新たに作成

In [1]:
import json

train = []
with open("data/TreePos_BI/train.json","r") as f:
    for line in f:
        train.append(json.loads(line))
for d in train:
    d["tree_pos"] = [d["tags"][i].split("_")[0] for i in range(len(d["tags"]))]
    d["ner_pos"] = [d["tags"][i].split("_")[1] for i in range(len(d["tags"]))]

train_includeBI = [d for d in train if list(set(d["ner_pos"])) != ["O"]]

# with open("data/TreePos_BI_small/50/train_50.json", 'w') as f:
#     for d in train_includeBI[0:50]:
#         f.write(json.dumps(d))
#         f.write("\n")

# with open("data/TreePos_BI_small/150/train_150.json", 'w') as f:
#     for d in train_includeBI[0:150]:
#         f.write(json.dumps(d))
#         f.write("\n")

# with open("data/TreePos_BI_small/500/train_500.json", 'w') as f:
#     for d in train_includeBI[0:500]:
#         f.write(json.dumps(d))
#         f.write("\n")

# データ拡張の作り直し

In [11]:
train_50 = train_includeBI[0:50]
train_150 = train_includeBI[0:150]
train_500 = train_includeBI[0:500]

In [40]:
# d= {'tokens': ['The',...,'.'],'tags': ['DT_O',...,'._O'],'tree_pos': ['DT',....'.'], 'ner_pos': ['O',...'O']}
def make_data2list_Simple(data):
    data_list = []
    for d in data:            
        new_sent = []
        before_tag = ""
        new_token = []
        for i in range(len(d["tags"])):

            if d["ner_pos"][i] == "O" or i == len(d["tokens"])-1:#固有表現タグがO　または　最後のトークンの時
                if d["ner_pos"][i] == before_tag:#1
                    new_token.append(d["tokens"][i])
                    new_sent.append([new_token,d["ner_pos"][i]])
                    new_token = []
                else:
                    new_token = []#2
                    new_token.append(d["tokens"][i])
                    new_sent.append([new_token,d["ner_pos"][i]])
                    new_token = []
                    
            else:#固有表現タグがBI#3
                if d["ner_pos"][i] == before_tag:
                    new_token.append(d["tokens"][i])
                    new_token = []
                else:#4
                    new_token.append(d["tokens"][i])
                    new_sent.append([new_token,d["ner_pos"][i]])
            before_tag = d["ner_pos"][i]
        data_list.append(new_sent)
    return data_list
# dev_list= [['Danish', 'rigsdaler'], 'other-currency'],[['='], 'O'],...


def make_data2list_TreePos(data):
    data_list = []
    for d in data:            
        new_sent = []
        before_tag = ""
        new_token = []
        new_tag = []
        for i in range(len(d["tags"])):

            if d["ner_pos"][i] == "O" or i == len(d["tokens"])-1:#固有表現タグがO　または　最後のトークンの時
                if d["ner_pos"][i] == before_tag:#1
                    new_token.append(d["tokens"][i])
                    new_tag.append(d["tags"][i])
                    
                    new_tag = list(set(new_tag))  #new_tagの重複削除
                    new_tag.sort()                #new_tagの整列
                    new_sent.append([new_token,"__".join(new_tag)])
                    
                    new_token = []
                    new_tag = []
                else:
                    new_token = []
                    new_tag = []
                    
                    new_token.append(d["tokens"][i])
                    new_tag.append(d["tags"][i])
                    
                    new_tag = list(set(new_tag))  #new_tagの重複削除
                    new_tag.sort()                #new_tagの整列
                    new_sent.append([new_token,"__".join(new_tag)])
                    
                    new_token = []
                    new_tag = []
                    
            else:#固有表現タグがBI
                if d["ner_pos"][i] == before_tag:
                    new_token.append(d["tokens"][i])
                    new_tag.append(d["tags"][i])
                    
                    new_token = []
                    new_tag = []
                else:
                    new_token.append(d["tokens"][i])
                    new_tag.append(d["tags"][i])
                    
                    new_tag = list(set(new_tag))  #new_tagの重複削除
                    new_tag.sort()                #new_tagの整列
                    new_sent.append([new_token,"__".join(new_tag)])#new_tagを__で結合
                    
            before_tag = d["ner_pos"][i]
        data_list.append(new_sent)
    return data_list

#交換先辞書の作成
def make_alterna_labels(data_list):
    label_list = set()
    for s in data_list:
        for t in s:
#             if t[1] != "O" and "_O" not in t[1][0]:
            label_list.add(t[1])
    
    label_list = list(label_list)
    alterna_labels = dict()
    for label in label_list:
        alterna_labels[label] = set()
    for s in data_list:
        for t in s:
            token = "_".join(t[0])
            tag = t[1]
            if tag in label_list:
                alterna_labels[tag].add(token)
    
    new_alterna_labels = dict()
    for k,v in alterna_labels.items():#候補が空白or1つのtagは削除
        if v==set() or len(v) == 1:
            pass
        else:
            new_alterna_labels[k] = v

    return new_alterna_labels


def make_alterna_labels_for_O(data_list):
    label_list = set()
    for s in data_list:
        for t in s:
            if t[1] == "O" or "_O" in t[1][0]:
                label_list.add(t[1])
    
    label_list = list(label_list)
    alterna_labels = dict()
    for label in label_list:
        alterna_labels[label] = set()
    for s in data_list:
        for t in s:
            token = "_".join(t[0])
            tag = t[1]
            if tag in label_list:
                alterna_labels[tag].add(token)
    
    new_alterna_labels = dict()
    for k,v in alterna_labels.items():#候補が空白or1つのtagは削除
        if v==set() or len(v) == 1:
            pass
        else:
            new_alterna_labels[k] = v

    return new_alterna_labels

In [39]:
import random
import json

parm_exchange = 0.5
parm_generate_per_s = 7#オリジナルデータの何倍データを作るか（生成後データにはオリジナルデータも含む）

def random_choice_token(token_set):
    n = random.randint(0,len(token_set)-1)
    token_list=list(token_set)
    return token_list[n]

def Simple_TreePos(datatype,data_dict,BI_or_O):
    #[[token_1,...token_n],tag],...[token_1,...token_n],tag]]状態のデータを作成
    data_simple = make_data2list_Simple(data_dict)#tagが固有表現情報
    data_treepos = make_data2list_TreePos(data_dict)#tagが品詞＋固有表現情報
    alterna_labels_simple = make_alterna_labels(data_simple)#固有表現情報のみに着目した交換先辞書
    alterna_labels_treepos = make_alterna_labels(data_treepos)#品詞＋固有表現情報の交換先辞書
    
    #加工箇所を同じにするため，SimpleとTreePosを同時に作成する
    data_simple_f_path = "data/Simple_"+BI_or_O+"_small/"+datatype+".json"
    data_treepos_f_path = "data/TreePos_"+BI_or_O+"_small/"+datatype+".json"
    memo_f_path = "log/TreePos_"+BI_or_O+"_small_"+datatype+"_memo.txt"

    with open(data_simple_f_path,"w",encoding="utf-8") as data_simple_f, \
    open(data_treepos_f_path,"w",encoding="utf-8") as data_treepos_f, \
    open(memo_f_path,"w",encoding="utf-8") as memo_f:
        
        for i in range(len(data_dict)):
            d = data_dict[i]
            write_d = dict()
            write_d["tokens"] = d["tokens"]
            write_d["tags"] = d["ner_pos"]
            data_simple_f.write(json.dumps(write_d))#Simpleにオリジナルデータを先に書込
            data_simple_f.write("\n")
            data_treepos_f.write(json.dumps(write_d))#TreePosにオリジナルデータを先に書込
            data_treepos_f.write("\n")
            
            cnt = 0
            roop_cnt = 0
            while cnt<parm_generate_per_s-1 and roop_cnt<30:
                roop_cnt += 1
                new_d_simple = dict()
                new_d_simple["tokens"] = []
                new_d_simple["tags"] = []

                new_d_treepos = dict()
                new_d_treepos["tokens"] = []
                new_d_treepos["tags"] = []
                old_tokens = []

                sentence_contain_change = False
                
                for token,tag_treepos in data_treepos[i]:
                    tag_simple = tag_treepos.split("_")[-1]
                    old_tokens.extend(token)
                    
                    #固有表現であり，厳格な候補であるtreeposに交換対象が存在するとき，50%の確率で入れ替えを発生
                    if  (tag_treepos in alterna_labels_treepos) and (random.random() >= parm_exchange):
                        if ((BI_or_O == "BI") and (tag_simple != "O")) or ((BI_or_O == "BI") and (tag_simple == "O")):

                            #固有表現情報に基づく交換候補を生成
                            token_simple = random_choice_token(alterna_labels_simple[tag_simple]-{"_".join(token)})
                            token_simple = token_simple.split("_")
                            #交換したtokenを記録
                            new_d_simple["tokens"].extend(token_simple)
                            #新しいtokenの数だけtagを追加
                            for _ in token_simple:
                                new_d_simple["tags"].append(tag_simple)

                            #品詞＋固有表現情報に基づく交換候補を生成
                            token_treepos = random_choice_token(alterna_labels_treepos[tag_treepos]-{"_".join(token)})
                            token_treepos = token_treepos.split("_")
                            #交換したtokenを記録
                            new_d_treepos["tokens"].extend(token_treepos)
                            #新しいtokenの数だけtagを追加
                            for _ in token_treepos:#
                                new_d_treepos["tags"].append(tag_simple)
                            sentence_contain_change = True#書換えフラグ

                    else:#交換しない固有表現タグ付きのtoken群（同じタグ唖連続している場合の人まとまり）
                        new_d_simple["tokens"].extend(token)
                        for _ in token:
                            new_d_simple["tags"].append(tag_simple)

                        new_d_treepos["tokens"].extend(token)
                        for _ in token:
                            new_d_treepos["tags"].append(tag_simple)

                if sentence_contain_change == True:#書換えが実行されたなら現在の一文をデータとして書出（改変前と同じ学習データの混入防止）
                    if (len(new_d_simple["tokens"]) == len(new_d_simple["tags"])) and (len(new_d_treepos["tokens"]) == len(new_d_treepos["tags"])):
                        data_simple_f.write(json.dumps(new_d_simple))
                        data_simple_f.write("\n")
                        data_treepos_f.write(json.dumps(new_d_treepos))
                        data_treepos_f.write("\n")

                        memo_f.write(str(i)+"\t"+" ".join(old_tokens)+"\n")#memoに変更前と変更後を記入
                        memo_f.write(" "*len(str(i))+"\t"+" ".join(new_d_simple["tokens"])+"\n")
                        memo_f.write(" "*len(str(i))+"\t"+" ".join(new_d_treepos["tokens"])+"\n")
                        memo_f.write("\n")
                        cnt+=1
                else:
                    pass
        memo_f.write("sentences augmented!!!! \n")

In [76]:
Simple_TreePos("train_50",train_50,"BI")
Simple_TreePos("train_150",train_150,"BI")
Simple_TreePos("train_500",train_500,"BI")

In [77]:
Simple_TreePos("train_50",train_50,"O")
Simple_TreePos("train_150",train_150,"O")
Simple_TreePos("train_500",train_500,"O")

## 11/28 ラベルごとの評価を観察

In [104]:
import json
import csv
import evaluate

In [105]:
seqeval = evaluate.load('seqeval')

In [ ]:
for n in [5,7,10,15,20]:
# for n in [3]:
    for task_name in ["f","Simple_BI_x2","Simple_O_x2","TreePos_BI_x2","TreePos_O_x2"]:
        print(n," ",task_name)

        references = []
        with open("data/f/test.json","r",encoding="utf-8") as f:
            for line in f:
                references.append(json.loads(line)["tags"])
                
        predictions = []
        with open("result/epochs"+str(n)+"/"+task_name+"/predictions.txt","r",encoding="utf-8") as f:
            for line in f:
                predictions.append(line.replace("\n","").split(" "))

        label_set = set()
        for l in references+predictions:
            for label in list(set(l)):
                label_set.add(label)

        label_set.remove("O")
        label_d = dict()
        for k in label_set:
            label_d[k] = "L"+k.replace("/","_").replace("-","_")

        for l in references:
            for i in range(len(l)):
                if l[i] in label_d:
                    l[i] = label_d[l[i]]

        for l in predictions:
            for i in range(len(l)):
                if l[i] in label_d:
                    l[i] = label_d[l[i]]
        
        results = seqeval.compute(predictions=predictions, references=references)
        print(1)
        results_overall = {}
        results_overall['overall_precision']=results['overall_precision']
        results_overall['overall_recall']=results['overall_recall']
        results_overall['overall_f1']=results['overall_f1']
        results_overall['overall_accuracy']=results['overall_accuracy']
        
        del results['overall_precision']
        del results['overall_recall']
        del results['overall_f1']
        del results['overall_accuracy']#項目に含まれる数値のみを値とするkeyを削除
        
        with open("result/all_label_result/epoch"+str(n)+"_"+task_name+"_overall.csv", 'w') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=["overall_precision","overall_recall","overall_f1","overall_accuracy"],lineterminator='\n')
            writer.writeheader()
            writer.writerow(results_overall)
            
        print(2)
        result_csv = []
        for k in results.keys():
            results[k]["label"] = k
            result_csv.append(results[k])

        with open("result/all_label_result/epoch"+str(n)+"_"+task_name+"_each_label.csv", 'w') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=["label","precision","recall","f1","number"],lineterminator='\n')
            writer.writeheader()
            writer.writerows([results[data] for data in results])

!rundll32 user32.dll,MessageBeep

## 各タグのサイズに配慮したSML

In [44]:
import json

train = []#131719
with open("data/TreePos_BI/train.json","r") as f:
    for line in f:
        train.append(json.loads(line))
for d in train:
    d["tree_pos"] = [d["tags"][i].split("_")[0] for i in range(len(d["tags"]))]
    d["ner_pos"] = [d["tags"][i].split("_")[1] for i in range(len(d["tags"]))]

train_includeBI = [d for d in train if list(set(d["ner_pos"])) != ["O"]]#113983

In [45]:
labels = ["art-broadcastprogram","art-film","art-music","art-other","art-painting","art-writtenart","building-airport","building-hospital","building-hotel","building-library","building-other","building-restaurant","building-sportsfacility","building-theater","event-attack/battle/war/militaryconflict","event-disaster","event-election","event-other","event-protest","event-sportsevent","location-GPE","location-bodiesofwater","location-island","location-mountain","location-other","location-park","location-road/railway/highway/transit","organization-company","organization-education","organization-government/governmentagency","organization-media/newspaper","organization-other","organization-politicalparty","organization-religion","organization-showorganization","organization-sportsleague","organization-sportsteam","other-astronomything","other-award","other-biologything","other-chemicalthing","other-currency","other-disease","other-educationaldegree","other-god","other-language","other-law","other-livingthing","other-medical","person-actor","person-artist/author","person-athlete","person-director","person-other","person-politician","person-scholar","person-soldier","product-airplane","product-car","product-food","product-game","product-other","product-ship","product-software","product-train","product-weapon"]

In [46]:
# 66種のラベルがkey ラベルを含むtrainの各要素を含むリストが値
labeled_data = []
for label in labels:
    new_l = []
    for d in train_includeBI:
        if label in set(d["ner_pos"]):
            new_l.append(d)
    labeled_data.append(new_l)#listに保存
#     with open("data/f_label/"+label+"__"+str(len(new_l))+".json","w",encoding="utf-8") as f:#種類別ファイルに保存
#         for d in new_l:
#             f.write(json.dumps(d))
#             f.write("\n")

In [47]:
#各ラベルごとに10分の1のスケールとしたデータを作成
import random
labeled_data_elected = []#21548

n = 10
for i in range(len(labeled_data)):
    choice_num = int(len(labeled_data[i])/n)
    choice_list = random.sample(labeled_data[i],choice_num)
    labeled_data_elected.extend(choice_list)

In [68]:
import random
import json

parm_exchange = 0.5
parm_generate_per_s = 7#オリジナルデータの何倍データを作るか（生成後データにはオリジナルデータも含む）

def random_choice_token(token_set):
    n = random.randint(0,len(token_set)-1)
    token_list=list(token_set)
    return token_list[n]

def Simple_TreePos(datatype,data_dict,BI_or_O):
    print(1)
    #[[token_1,...token_n],tag],...[token_1,...token_n],tag]]状態のデータを作成
    data_simple = make_data2list_Simple(data_dict)#tagが固有表現情報
    data_treepos = make_data2list_TreePos(data_dict)#tagが品詞＋固有表現情報
    alterna_labels_simple = make_alterna_labels(data_simple)#固有表現情報のみに着目した交換先辞書
    alterna_labels_treepos = make_alterna_labels(data_treepos)#品詞＋固有表現情報の交換先辞書
    print(2)
    #加工箇所を同じにするため，SimpleとTreePosを同時に作成する
    data_simple_f_path = "data/Simple_"+BI_or_O+"_small/"+datatype+".json"
    data_treepos_f_path = "data/TreePos_"+BI_or_O+"_small/"+datatype+".json"
    memo_f_path = "log/TreePos_"+BI_or_O+"_small_"+datatype+"_memo.txt"
    print(3)
    with open(data_simple_f_path,"w",encoding="utf-8") as data_simple_f, \
    open(data_treepos_f_path,"w",encoding="utf-8") as data_treepos_f, \
    open(memo_f_path,"w",encoding="utf-8") as memo_f:
        
        for i in range(len(data_dict)):
            print('\r%d / %d' %(i, len(data_dict)), end='')
            
            d = data_dict[i]
            write_d = dict()
            write_d["tokens"] = d["tokens"]
            write_d["tags"] = d["ner_pos"]
            data_simple_f.write(json.dumps(write_d))#Simpleにオリジナルデータを先に書込
            data_simple_f.write("\n")
            data_treepos_f.write(json.dumps(write_d))#TreePosにオリジナルデータを先に書込
            data_treepos_f.write("\n")
            
            cnt = 0
            roop_cnt = 0
            while cnt<parm_generate_per_s-1 and roop_cnt<30:
                
                roop_cnt += 1
                new_d_simple = dict()
                new_d_simple["tokens"] = []
                new_d_simple["tags"] = []

                new_d_treepos = dict()
                new_d_treepos["tokens"] = []
                new_d_treepos["tags"] = []
                old_tokens = []

                sentence_contain_change = False
                
                for token,tag_treepos in data_treepos[i]:
                    tag_simple = tag_treepos.split("_")[-1]
                    old_tokens.extend(token)
                    
                    #固有表現であり，厳格な候補であるtreeposに交換対象が存在するとき，50%の確率で入れ替えを発生
                    if  (tag_treepos in alterna_labels_treepos) and (random.random() >= parm_exchange):
                        if ((BI_or_O == "BI") and (tag_simple != "O")) or ((BI_or_O == "BI") and (tag_simple == "O")):

                            #固有表現情報に基づく交換候補を生成
                            token_simple = random_choice_token(alterna_labels_simple[tag_simple]-{"_".join(token)})
                            token_simple = token_simple.split("_")
                            #交換したtokenを記録
                            new_d_simple["tokens"].extend(token_simple)
                            #新しいtokenの数だけtagを追加
                            for _ in token_simple:
                                new_d_simple["tags"].append(tag_simple)

                            #品詞＋固有表現情報に基づく交換候補を生成
                            token_treepos = random_choice_token(alterna_labels_treepos[tag_treepos]-{"_".join(token)})
                            token_treepos = token_treepos.split("_")
                            #交換したtokenを記録
                            new_d_treepos["tokens"].extend(token_treepos)
                            #新しいtokenの数だけtagを追加
                            for _ in token_treepos:#
                                new_d_treepos["tags"].append(tag_simple)
                            sentence_contain_change = True#書換えフラグ

                    else:#交換しない固有表現タグ付きのtoken群（同じタグ唖連続している場合の人まとまり）
                        new_d_simple["tokens"].extend(token)
                        for _ in token:
                            new_d_simple["tags"].append(tag_simple)

                        new_d_treepos["tokens"].extend(token)
                        for _ in token:
                            new_d_treepos["tags"].append(tag_simple)

                if sentence_contain_change == True:#書換えが実行されたなら現在の一文をデータとして書出（改変前と同じ学習データの混入防止）
                    if (len(new_d_simple["tokens"]) == len(new_d_simple["tags"])) and (len(new_d_treepos["tokens"]) == len(new_d_treepos["tags"])):
                        data_simple_f.write(json.dumps(new_d_simple))
                        data_simple_f.write("\n")
                        data_treepos_f.write(json.dumps(new_d_treepos))
                        data_treepos_f.write("\n")

                        memo_f.write(str(i)+"\t"+" ".join(old_tokens)+"\n")#memoに変更前と変更後を記入
                        memo_f.write(" "*len(str(i))+"\t"+" ".join(new_d_simple["tokens"])+"\n")
                        memo_f.write(" "*len(str(i))+"\t"+" ".join(new_d_treepos["tokens"])+"\n")
                        memo_f.write("\n")
                        cnt+=1
                else:
                    pass
        memo_f.write("sentences augmented!!!! \n")

In [69]:
Simple_TreePos("labeled_small",labeled_data_elected,"BI")

1
2
3
21547 / 21548

In [70]:
Simple_TreePos("labeled_small",labeled_data_elected,"O")

1
2
3
21547 / 21548

In [53]:
data_dict = labeled_data_elected

In [55]:
data_simple = make_data2list_Simple(data_dict)#tagが固有表現情報
print(1)
data_treepos = make_data2list_TreePos(data_dict)#tagが品詞＋固有表現情報
print(1)
alterna_labels_simple = make_alterna_labels(data_simple)#固有表現情報のみに着目した交換先辞書
print(1)
alterna_labels_treepos = make_alterna_labels(data_treepos)#品詞＋固有表現情報の交換先辞書

1
1
1


In [62]:
alterna_labels_treepos

{'Chick-fil-A', 'The_Anchor', 'The_Cheesecake', 'The_French', 'The_Mad'}